# Houston, TX Simulations
8760 rows per simulation<br>
50 files (removed cell-in-cell)

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
import pprint
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

foot-candles --> Lux (multiply by 10.7642)

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

In [2]:
# 2018 Egrid Costs (elec $/kWh, gas $/therm), change values per city
gas = 1.11
elec = 0.112

---

## Combine all csv files in directory

In [3]:
# Create variable for files in directory
files = [f for f in os.listdir("data/hou/") if f.endswith(".csv")]

#files

In [4]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_data = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/hou/" + file)
    all_data = pd.concat([all_data, df])
    
all_data

# 438000  rows, 73 columns

,Scenario,Date/Time,LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly),LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Daylight Illuminance Setpoint Exceeded Time [hr](Hourly),LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Glare Index [](Hourly),LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Glare Index Setpoint Exceeded Time [hr](Hourly),WINDOW_LDF1_1_BOT.UNIT1:Daylighting Window Reference Point 1 Illuminance [lux](Hourly),WINDOW_LDF1_1_BOT.UNIT1:Daylighting Window Reference Point 1 View Luminance [cd/m2](Hourly),WINDOW_LDF1_1_TOP.UNIT1:Daylighting Window Reference Point 1 Illuminance [lux](Hourly),WINDOW_LDF1_1_TOP.UNIT1:Daylighting Window Reference Point 1 View Luminance [cd/m2](Hourly),...,WINDOW_SDL2_1_BOT.UNIT1:Daylighting Window Reference Point 1 View Luminance [cd/m2](Hourly),WINDOW_SDL2_1_TOP.UNIT1:Daylighting Window Reference Point 1 Illuminance [lux](Hourly),WINDOW_SDL2_1_TOP.UNIT1:Daylighting Window Reference Point 1 View Luminance [cd/m2](Hourly),WINDOW_SDL2_2_BOT.UNIT1:Daylighting Window Reference Point 1 Illuminance [lux](Hourly),WINDOW_SDL2_2_BOT.UNIT1:Daylighting Window Reference Point 1 View Luminance [cd/m2](Hourly),WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 1 Illuminance [lux](Hourly),WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 1 View Luminance [cd/m2](Hourly),SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly),MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly),DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly)
0,hou_102_102_base,01/01 01:00:00,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,612374.44620,1.862881e+07,0.0
1,hou_102_102_base,01/01 02:00:00,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,669016.25430,2.035189e+07,0.0
2,hou_102_102_base,01/01 03:00:00,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,716355.20410,2.179197e+07,0.0
3,hou_102_102_base,01/01 04:00:00,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,761761.05950,2.317324e+07,0.0
4,hou_102_102_base,01/01 05:00:00,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,783507.59790,2.383479e+07,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,hou_2154_BBG-VB75_102,12/31 20:00:00,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000e+00,0.0
8756,hou_2154_BBG-VB75_102,12/31 21:00:00,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000e+00,0.0
8757,hou_2154_BBG-VB75_102,12/31 22:00:00,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12311.68068,3.745289e+05,0.0
8758,hou_2154_BBG-VB75_102,12/31 23:00:00,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,153460.86920,4.668375e+06,0.0


In [5]:
all_data.dtypes

Scenario                                                                                                                    object
Date/Time                                                                                                                   object
LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)                                   float64
LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Daylight Illuminance Setpoint Exceeded Time [hr](Hourly)    float64
LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Glare Index [](Hourly)                                      float64
                                                                                                                            ...   
WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 1 Illuminance [lux](Hourly)                                     float64
WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 1 View Luminance [cd/m2]

In [6]:
# print(all_data)
# is_NaN = all_data. isnull()
# row_has_NaN = is_NaN. any(axis=1)
# rows_with_NaN = all_data[row_has_NaN]
# print(rows_with_NaN)

In [7]:
# Get wanted columns
all_data1 = all_data[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) "]]

#all_data1

In [8]:
all_data1 = all_data1.dropna().reset_index(drop=True)

# all_data1

#438000  rows, 73 columns

-----

## Get just the wanted data (columns)

In [9]:
# Rename Columns
all_data2 = all_data1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#all_data2

#438000  rows, 6 columns

-----

## Get annual energy data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>


In [10]:
# Copy dataframe to modify and leave original df intact
get_annual = all_data2.copy(deep=True)

#get_annual

In [11]:
# Convert Joules to KBtu in dataframe
get_annual["FanEnergy(kBtu)"] = get_annual["FanEnergy[J](Hourly)"] * .00000094781712
get_annual["HeatingEnergy(kBtu)"] = get_annual["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual["CoolingEnergy(kBtu)"] = get_annual["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_annual

In [12]:
# Drop columns
get_annual.drop(columns = ["Illuminance(lux)", "FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual

---

## Get annual cost data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>
Add cost multiplier, variable needs to be defined above

In [13]:
# Get annual costs
get_annual_costs = all_data2.copy(deep=True)

In [14]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs["FanEnergy($)"] = (get_annual_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_annual_costs["HeatingEnergy($)"] = ((get_annual_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_annual_costs["CoolingEnergy($)"] = (get_annual_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual_costs

In [15]:
# Drop columns
get_annual_costs.drop(columns = ["Illuminance(lux)", "FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual_costs

---

## Combine annual energy and cost data
Combine two dfs<br>
Group by scenario <br>
Export combine file

In [16]:
# Combine data for energy and cost
allAnnualData = pd.merge(get_annual, get_annual_costs, how="inner" )

#allAnnualData

In [17]:
# Add energy and cost totals
totalEnergy = allAnnualData["FanEnergy(kBtu)"] + allAnnualData["HeatingEnergy(kBtu)"] + allAnnualData["CoolingEnergy(kBtu)"]
allAnnualData["AnnualEnergy(kBtu)"] = totalEnergy

totalCost = allAnnualData["FanEnergy($)"] + allAnnualData["HeatingEnergy($)"] + allAnnualData["CoolingEnergy($)"]
allAnnualData["AnnualCost($)"] = totalCost

allAnnualData

,Scenario,Date_Time,FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),FanEnergy($),HeatingEnergy($),CoolingEnergy($),AnnualEnergy(kBtu),AnnualCost($)
0,hou_102_102_base,01/01 01:00:00,0.580419,17.656705,0.0,0.019052,0.195989,0.0,18.237124,0.215041
1,hou_102_102_base,01/01 02:00:00,0.634105,19.289868,0.0,0.020814,0.214118,0.0,19.923973,0.234931
2,hou_102_102_base,01/01 03:00:00,0.678974,20.654801,0.0,0.022287,0.229268,0.0,21.333775,0.251555
3,hou_102_102_base,01/01 04:00:00,0.722010,21.963996,0.0,0.023699,0.243800,0.0,22.686007,0.267500
4,hou_102_102_base,01/01 05:00:00,0.742622,22.591018,0.0,0.024376,0.250760,0.0,23.333640,0.275136
...,...,...,...,...,...,...,...,...,...,...
437995,hou_2154_BBG-VB75_102,12/31 20:00:00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
437996,hou_2154_BBG-VB75_102,12/31 21:00:00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
437997,hou_2154_BBG-VB75_102,12/31 22:00:00,0.011669,0.354985,0.0,0.000383,0.003940,0.0,0.366654,0.004323
437998,hou_2154_BBG-VB75_102,12/31 23:00:00,0.145453,4.424765,0.0,0.004774,0.049115,0.0,4.570218,0.053889


---
## Get scenario names 
Get the names to a list<br>
Export to file

In [18]:
# # Create new df of just scenario names
# # Export as a csv for javascript filter
# scenarios = allAnnualData["Scenario"].unique()
# print(scenarios)

In [19]:
# scenarios.tofile("data/final/hou_scenarios.csv", sep=",", format="%s")

---
## Group and export
Group dataframe by scenario<br>
Export to file

In [20]:
# Group by scenario
annualData = allAnnualData.groupby(["Scenario"])

annualData.sum().round(2)

,FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),FanEnergy($),HeatingEnergy($),CoolingEnergy($),AnnualEnergy(kBtu),AnnualCost($)
Scenario,,,,,,,,
hou_102_102_1VB-15,5582.42,19119.45,31483.84,183.24,212.23,1033.42,56185.72,1428.89
hou_102_102_1VB-30,5576.72,19096.21,31446.43,183.05,211.97,1032.20,56119.35,1427.21
hou_102_102_1VB-45,5477.17,19302.83,30795.02,179.78,214.26,1010.81,55575.02,1404.86
hou_102_102_1VB-60,5263.61,19789.22,29396.71,172.77,219.66,964.92,54449.55,1357.35
hou_102_102_1VB-75,4969.22,20527.14,27445.38,163.11,227.85,900.87,52941.74,1291.83
hou_102_102_1VB0,5527.19,19289.60,31119.75,181.42,214.11,1021.47,55936.54,1417.01
hou_102_102_1VB15,5426.21,19537.35,30456.62,178.11,216.86,999.71,55420.18,1394.68
hou_102_102_1VB30,5297.63,19833.07,29611.02,173.89,220.15,971.95,54741.72,1365.99
hou_102_102_1VB45,5141.95,20178.98,28585.10,168.78,223.99,938.28,53906.03,1331.04


In [21]:
# Export grouped file to csv
annualData.sum().round(2).to_csv("../Scraper_Output/hou_annual_data.csv", header=True, index=True)

-----

## Get daily and monthly energy data
Group by scenario <br>
Groups by month <br>
Change units from Joules to therms and kWh

In [22]:
# Copy dataframe to modify and leave original df intact
get_monthly = all_data2.copy(deep=True)

#get_monthly

In [23]:
# Convert Joules to kWh and therms in new dataframe
get_monthly["FanEnergy(kWh)"] = get_monthly["FanEnergy[J](Hourly)"] * .000000277777778
get_monthly["HeatingEnergy(therms)"] = get_monthly["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_monthly["CoolingEnergy(kWh)"] = get_monthly["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_monthly

In [24]:
# Drop columns
get_monthly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

get_monthly

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh)
0,hou_102_102_base,01/01 01:00:00,0.0,0.170104,0.176567,0.0
1,hou_102_102_base,01/01 02:00:00,0.0,0.185838,0.192899,0.0
2,hou_102_102_base,01/01 03:00:00,0.0,0.198988,0.206548,0.0
3,hou_102_102_base,01/01 04:00:00,0.0,0.211600,0.219640,0.0
4,hou_102_102_base,01/01 05:00:00,0.0,0.217641,0.225910,0.0
...,...,...,...,...,...,...
437995,hou_2154_BBG-VB75_102,12/31 20:00:00,0.0,0.000000,0.000000,0.0
437996,hou_2154_BBG-VB75_102,12/31 21:00:00,0.0,0.000000,0.000000,0.0
437997,hou_2154_BBG-VB75_102,12/31 22:00:00,0.0,0.003420,0.003550,0.0
437998,hou_2154_BBG-VB75_102,12/31 23:00:00,0.0,0.042628,0.044248,0.0


In [25]:
# get_monthly["Month"] = get_monthly["Date_Time"].apply(lambda x: str(x)[0:3])

# get_monthly

## Get monthly cost data

Group by scenario <br>
Groups by month <br>
Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [26]:
# Copy dataframe to modify and leave original df intact
get_monthly_costs = all_data2.copy(deep=True)

#get_monthly

In [27]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_monthly_costs["FanEnergy($)"] = (get_monthly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_monthly_costs["HeatingEnergy($)"] = ((get_monthly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_monthly_costs["CoolingEnergy($)"] = (get_monthly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly_costs

In [28]:
# Drop columns
get_monthly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

get_monthly_costs

,Scenario,Date_Time,Illuminance(lux),FanEnergy($),HeatingEnergy($),CoolingEnergy($)
0,hou_102_102_base,01/01 01:00:00,0.0,0.019052,0.195989,0.0
1,hou_102_102_base,01/01 02:00:00,0.0,0.020814,0.214118,0.0
2,hou_102_102_base,01/01 03:00:00,0.0,0.022287,0.229268,0.0
3,hou_102_102_base,01/01 04:00:00,0.0,0.023699,0.243800,0.0
4,hou_102_102_base,01/01 05:00:00,0.0,0.024376,0.250760,0.0
...,...,...,...,...,...,...
437995,hou_2154_BBG-VB75_102,12/31 20:00:00,0.0,0.000000,0.000000,0.0
437996,hou_2154_BBG-VB75_102,12/31 21:00:00,0.0,0.000000,0.000000,0.0
437997,hou_2154_BBG-VB75_102,12/31 22:00:00,0.0,0.000383,0.003940,0.0
437998,hou_2154_BBG-VB75_102,12/31 23:00:00,0.0,0.004774,0.049115,0.0


In [29]:
# get_monthly_costs["Month"] = get_monthly_costs["Date_Time"].apply(lambda x: str(x)[0:3])

# get_monthly_costs

---

## Combine monthly energy and cost data
Combine two dfs<br>
Create Month Column<br>
Group by scenario <br>
Export combine file

In [30]:
# Combine data for energy and cost
allMonthlyData = pd.merge(get_monthly, get_monthly_costs, how="inner" )

#allMonthlyData

In [31]:
# Add energy and cost totals
totalEnergyMonthly = allMonthlyData["FanEnergy(kWh)"] + allMonthlyData["HeatingEnergy(therms)"] + allMonthlyData["CoolingEnergy(kWh)"]
allMonthlyData["MonthlyEnergy(kBtu)"] = totalEnergyMonthly

totalCostMonthly = allMonthlyData["FanEnergy($)"] + allMonthlyData["HeatingEnergy($)"] + allMonthlyData["CoolingEnergy($)"]
allMonthlyData["MonthlyCost($)"] = totalCostMonthly

averageIlluminanceMonthly = allMonthlyData["Illuminance(lux)"].mean()

#allMonthlyData

In [32]:
allMonthlyData["Month"] = allMonthlyData.Date_Time.str.slice(0,3)

allMonthlyData

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanEnergy($),HeatingEnergy($),CoolingEnergy($),MonthlyEnergy(kBtu),MonthlyCost($),Month
0,hou_102_102_base,01/01 01:00:00,0.0,0.170104,0.176567,0.0,0.019052,0.195989,0.0,0.346671,0.215041,01
1,hou_102_102_base,01/01 02:00:00,0.0,0.185838,0.192899,0.0,0.020814,0.214118,0.0,0.378737,0.234931,01
2,hou_102_102_base,01/01 03:00:00,0.0,0.198988,0.206548,0.0,0.022287,0.229268,0.0,0.405536,0.251555,01
3,hou_102_102_base,01/01 04:00:00,0.0,0.211600,0.219640,0.0,0.023699,0.243800,0.0,0.431240,0.267500,01
4,hou_102_102_base,01/01 05:00:00,0.0,0.217641,0.225910,0.0,0.024376,0.250760,0.0,0.443551,0.275136,01
...,...,...,...,...,...,...,...,...,...,...,...,...
437995,hou_2154_BBG-VB75_102,12/31 20:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,12
437996,hou_2154_BBG-VB75_102,12/31 21:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,12
437997,hou_2154_BBG-VB75_102,12/31 22:00:00,0.0,0.003420,0.003550,0.0,0.000383,0.003940,0.0,0.006970,0.004323,12
437998,hou_2154_BBG-VB75_102,12/31 23:00:00,0.0,0.042628,0.044248,0.0,0.004774,0.049115,0.0,0.086876,0.053889,12


In [33]:
# Drop Illuminance column...don't need the totals
allMonthlyData2 = allMonthlyData.drop(["Illuminance(lux)", "MonthlyEnergy(kBtu)" ], axis=1)
allMonthlyData2

,Scenario,Date_Time,FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanEnergy($),HeatingEnergy($),CoolingEnergy($),MonthlyCost($),Month
0,hou_102_102_base,01/01 01:00:00,0.170104,0.176567,0.0,0.019052,0.195989,0.0,0.215041,01
1,hou_102_102_base,01/01 02:00:00,0.185838,0.192899,0.0,0.020814,0.214118,0.0,0.234931,01
2,hou_102_102_base,01/01 03:00:00,0.198988,0.206548,0.0,0.022287,0.229268,0.0,0.251555,01
3,hou_102_102_base,01/01 04:00:00,0.211600,0.219640,0.0,0.023699,0.243800,0.0,0.267500,01
4,hou_102_102_base,01/01 05:00:00,0.217641,0.225910,0.0,0.024376,0.250760,0.0,0.275136,01
...,...,...,...,...,...,...,...,...,...,...
437995,hou_2154_BBG-VB75_102,12/31 20:00:00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,12
437996,hou_2154_BBG-VB75_102,12/31 21:00:00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,12
437997,hou_2154_BBG-VB75_102,12/31 22:00:00,0.003420,0.003550,0.0,0.000383,0.003940,0.0,0.004323,12
437998,hou_2154_BBG-VB75_102,12/31 23:00:00,0.042628,0.044248,0.0,0.004774,0.049115,0.0,0.053889,12


In [34]:
# Group by scenario and date/time for export to graphing
MonthlyData = allMonthlyData2.groupby(["Scenario", "Date_Time"])

MonthlyData.sum()

FanEnergy(kWh)  HeatingEnergy(therms)  \
Scenario              Date_Time                                                 
hou_102_102_1VB-15     01/01  01:00:00        0.161813               0.167961   
                       01/01  02:00:00        0.177439               0.184181   
                       01/01  03:00:00        0.190509               0.197748   
                       01/01  04:00:00        0.202978               0.210690   
                       01/01  05:00:00        0.208936               0.216875   
...                                                ...                    ...   
hou_2154_BBG-VB75_102  12/31  20:00:00        0.000000               0.000000   
                       12/31  21:00:00        0.000000               0.000000   
                       12/31  22:00:00        0.003420               0.003550   
                       12/31  23:00:00        0.042628               0.044248   
                       12/31  24:00:00        0.069644               0.072290   

                                        CoolingEnergy(kWh)  FanEnergy($)  \
Scenario              Date_Time                                            
hou_102_102_1VB-15     01/01  01:00:00                 0.0      0.018123   
                       01/01  02:00:00                 0.0      0.019873   
                       01/01  03:00:00                 0.0      0.021337   
                       01/01  04:00:00                 0.0      0.022734   
                       01/01  05:00:00                 0.0      0.023401   
...                                                    ...           ...   
hou_2154_BBG-VB75_102  12/31  20:00:00                 0.0      0.000000   
                       12/31  21:00:00                 0.0      0.000000   
                       12/31  22:00:00                 0.0      0.000383   
                       12/31  23:00:00                 0.0      0.004774   
                       12/31  24:00:00                 0.0      0.007800   

                                        HeatingEnergy($)  CoolingEnergy($)  \
Scenario              Date_Time                                              
hou_102_102_1VB-15     01/01  01:00:00          0.186436               0.0   
                       01/01  02:00:00          0.204440               0.0   
                       01/01  03:00:00          0.219500               0.0   
                       01/01  04:00:00          0.233866               0.0   
                       01/01  05:00:00          0.240731               0.0   
...                                                  ...               ...   
hou_2154_BBG-VB75_102  12/31  20:00:00          0.000000               0.0   
                       12/31  21:00:00          0.000000               0.0   
                       12/31  22:00:00          0.003940               0.0   
                       12/31  23:00:00          0.049115               0.0   
                       12/31  24:00:00          0.080241               0.0   

                                        MonthlyCost($)  
Scenario              Date_Time                         
hou_102_102_1VB-15     01/01  01:00:00        0.204560  
                       01/01  02:00:00        0.224314  
                       01/01  03:00:00        0.240837  
                       01/01  04:00:00        0.256599  
                       01/01  05:00:00        0.264132  
...                                                ...  
hou_2154_BBG-VB75_102  12/31  20:00:00        0.000000  
                       12/31  21:00:00        0.000000  
                       12/31  22:00:00        0.004323  
                       12/31  23:00:00        0.053889  
                       12/31  24:00:00        0.088042  

[438000 rows x 7 columns]

In [35]:
# Export grouped file to csv (used before grouped by month)
MonthlyData.sum().round(2).to_csv("../Scraper_Output/hou_monthly_data.csv", header=True, index=True)

In [36]:
# # Export grouped file to json, issues with exporting grouped csv
# MonthlyData.sum().round(2).to_json("data/final/hou_monthly_data.json")

In [37]:
# Group by scenario and date/time for export to graphing
MonthlyData2 = allMonthlyData2.groupby(["Scenario", "Month"])

MonthlyData2.sum()

FanEnergy(kWh)  HeatingEnergy(therms)  \
Scenario              Month                                          
hou_102_102_1VB-15     01         64.996867              57.586593   
                       02         61.847019              42.448752   
                       03         64.004196              21.491571   
                       04         89.162600               3.717218   
                       05        151.687971               0.075803   
...                                     ...                    ...   
hou_2154_BBG-VB75_102  08        199.688039               0.000000   
                       09        158.490742               0.000000   
                       10         61.109673               5.703230   
                       11         40.816589              19.004666   
                       12         53.569678              50.454338   

                             CoolingEnergy(kWh)  FanEnergy($)  \
Scenario              Month                                     
hou_102_102_1VB-15     01             53.852839      7.279649   
                       02            121.882904      6.926866   
                       03            254.547483      7.168470   
                       04            522.217173      9.986211   
                       05            949.164785     16.989053   
...                                         ...           ...   
hou_2154_BBG-VB75_102  08           1313.549256     22.365060   
                       09           1022.428631     17.750963   
                       10            345.880530      6.844283   
                       11            137.283017      4.571458   
                       12             29.915544      5.999804   

                             HeatingEnergy($)  CoolingEnergy($)  \
Scenario              Month                                       
hou_102_102_1VB-15     01           63.921118          6.031518   
                       02           47.118115         13.650885   
                       03           23.855644         28.509318   
                       04            4.126112         58.488323   
                       05            0.084141        106.306456   
...                                       ...               ...   
hou_2154_BBG-VB75_102  08            0.000000        147.117517   
                       09            0.000000        114.512007   
                       10            6.330586         38.738619   
                       11           21.095179         15.375698   
                       12           56.004315          3.350541   

                             MonthlyCost($)  
Scenario              Month                  
hou_102_102_1VB-15     01         77.232285  
                       02         67.695866  
                       03         59.533432  
                       04         72.600646  
                       05        123.379650  
...                                     ...  
hou_2154_BBG-VB75_102  08        169.482577  
                       09        132.262970  
                       10         51.913489  
                       11         41.042335  
                       12         65.354660  

[600 rows x 7 columns]

In [38]:
# Export grouped file to csv
MonthlyData2.sum().round(2).to_csv("../Scraper_Output/hou_monthly2_data.csv", header=True, index=True)

## Get monthly average illuminance
Get just the illuminance column<br>
Group by scenario <br>
Groups by month <br>

In [39]:
Illuminance = allMonthlyData[["Scenario", "Date_Time", "Illuminance(lux)", "Month"]]
Illuminance

,Scenario,Date_Time,Illuminance(lux),Month
0,hou_102_102_base,01/01 01:00:00,0.0,01
1,hou_102_102_base,01/01 02:00:00,0.0,01
2,hou_102_102_base,01/01 03:00:00,0.0,01
3,hou_102_102_base,01/01 04:00:00,0.0,01
4,hou_102_102_base,01/01 05:00:00,0.0,01
...,...,...,...,...
437995,hou_2154_BBG-VB75_102,12/31 20:00:00,0.0,12
437996,hou_2154_BBG-VB75_102,12/31 21:00:00,0.0,12
437997,hou_2154_BBG-VB75_102,12/31 22:00:00,0.0,12
437998,hou_2154_BBG-VB75_102,12/31 23:00:00,0.0,12


In [40]:
# Group by scenario
MonthlyIlluminance = Illuminance.groupby(["Scenario", "Date_Time"])

MonthlyIlluminance.mean()

Illuminance(lux)
Scenario              Date_Time                         
hou_102_102_1VB-15     01/01  01:00:00               0.0
                       01/01  02:00:00               0.0
                       01/01  03:00:00               0.0
                       01/01  04:00:00               0.0
                       01/01  05:00:00               0.0
...                                                  ...
hou_2154_BBG-VB75_102  12/31  20:00:00               0.0
                       12/31  21:00:00               0.0
                       12/31  22:00:00               0.0
                       12/31  23:00:00               0.0
                       12/31  24:00:00               0.0

[438000 rows x 1 columns]

In [41]:
# Export grouped file to csv
MonthlyIlluminance.mean().round(2).to_csv("../Scraper_Output/hou_monthly_illuminance.csv", header=True, index=True)

In [42]:
# # Export grouped file to json, issues with exporting grouped csv
# MonthlyIlluminance.mean().round(2).to_json("data/final/hou_monthly_illuminance.json")

In [43]:
# Group by scenario
MonthlyIlluminance2 = Illuminance.groupby(["Scenario", "Month"])

MonthlyIlluminance2.mean()

Illuminance(lux)
Scenario              Month                  
hou_102_102_1VB-15     01         1507.573519
                       02         1010.905731
                       03          723.674310
                       04          744.629761
                       05          655.237120
...                                       ...
hou_2154_BBG-VB75_102  08           16.085570
                       09           22.414874
                       10           27.465698
                       11           25.456639
                       12           22.509148

[600 rows x 1 columns]

In [44]:
# Export grouped file to csv
MonthlyIlluminance2.mean().round(2).to_csv("../Scraper_Output/hou_monthly_illuminance2.csv", header=True, index=True)

------

## Get hourly data

In [45]:
all_data3 = all_data1.copy(deep=True)

#all_data3

In [46]:
# Get wanted columns
# Include hourly illumanc
all_data3 = all_data3[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) "]]

#all_data3

In [47]:
# Rename Columns
get_hourly = all_data3.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance[lux](Hourly)",
                             "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#get_hourly

In [48]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_hourly["FanEnergy($)"] = (get_hourly["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_hourly["HeatingEnergy($)"] = ((get_hourly["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_hourly["CoolingEnergy($)"] = (get_hourly["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_hourly

In [49]:
# Drop columns
get_hourly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)


------

## Get hourly cost data

In [50]:
get_hourly_costs = all_data3.copy(deep=True)

#get_hourly_costs

In [51]:
# Rename Columns
get_hourly_costs = get_hourly_costs.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance[lux](Hourly)",
                             "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#get_hourly_costs

In [52]:
# Drop columns
get_hourly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)


## Combine hourly energy and cost data

In [53]:
# Combine data for energy and cost
allHourlyData = pd.merge(get_hourly, get_hourly_costs, how="inner" )

allHourlyData

,Scenario,Date_Time,Illuminance[lux](Hourly),FanEnergy($),HeatingEnergy($),CoolingEnergy($)
0,hou_102_102_base,01/01 01:00:00,0.0,0.019052,0.195989,0.0
1,hou_102_102_base,01/01 02:00:00,0.0,0.020814,0.214118,0.0
2,hou_102_102_base,01/01 03:00:00,0.0,0.022287,0.229268,0.0
3,hou_102_102_base,01/01 04:00:00,0.0,0.023699,0.243800,0.0
4,hou_102_102_base,01/01 05:00:00,0.0,0.024376,0.250760,0.0
...,...,...,...,...,...,...
437995,hou_2154_BBG-VB75_102,12/31 20:00:00,0.0,0.000000,0.000000,0.0
437996,hou_2154_BBG-VB75_102,12/31 21:00:00,0.0,0.000000,0.000000,0.0
437997,hou_2154_BBG-VB75_102,12/31 22:00:00,0.0,0.000383,0.003940,0.0
437998,hou_2154_BBG-VB75_102,12/31 23:00:00,0.0,0.004774,0.049115,0.0


In [54]:
# Export grouped file to csv for daily cost sums
allHourlyData.to_csv("../Scraper_Output/hou_daily_cost_sum.csv", header=True, index=False)